In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os
import zipfile
from google.oauth2 import service_account
from googleapiclient.discovery import build

# Set up the service account credentials
credentials = service_account.Credentials.from_service_account_file('/content/drive/MyDrive/Credentials.json')
drive_service = build('drive', 'v3', credentials=credentials)

# ID of the archive.zip file on Google Drive
file_id = '1g3J3n2QsbLwepYzGPVZfyOyMHtmHfQfA'

# Download the file
request = drive_service.files().get_media(fileId=file_id)
downloaded_file = request.execute()

with open('Gai_Dataset.zip', 'wb') as f:
    f.write(downloaded_file)

# Unzip the file
with zipfile.ZipFile('Gai_Dataset.zip', 'r') as zip_ref:
    zip_ref.extractall()

# Clean up: Delete the downloaded zip file
os.remove('Gai_Dataset.zip')

print('Extraction completed successfully.')


Extraction completed successfully.


In [ ]:
#=======================================================================================================================

In [ ]:
# Converting HDF5 (.h5) to Tflite format

In [ ]:
import tensorflow as tf

# Path to the .h5 model
h5_model_path = '/content/drive/MyDrive/GaiAppModelPath/GaiApp_Epoch10BestMobileNet.h5'

# Load the Keras model
keras_model = tf.keras.models.load_model(h5_model_path)

# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)
tflite_model = converter.convert()

# Save the TensorFlow Lite model
tflite_model_path = 'GaiApp_Epoch10.tflite'
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

print("Model converted and saved as:", tflite_model_path)


Model converted and saved as: GaiApp_Epoch10.tflite


In [ ]:
#EVALUATION TRAINING,TESTING,VALIDATION

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array, load_img

# Constants
IMAGE_SIZE1 = 240  # Height of the Image
IMAGE_SIZE2 = 320  # Width of the Image

# Specify the directory paths for the data
root_dir = "Gai_Dataset/"
train_dir = f"{root_dir}train/"

# Collect the class names
class_names = sorted(os.listdir(train_dir))
print(class_names)
print(len(class_names))


# Load the training dataset
def load_dataset(root_path, class_names, trim=None):
    X_data = []
    y_data = []
    for class_idx, class_name in enumerate(class_names):
        class_path = os.path.join(root_path, class_name)
        images = os.listdir(class_path)
        if trim:
            images = images[:trim]
        for image_name in images:
            image_path = os.path.join(class_path, image_name)
            image = load_img(image_path, target_size=(IMAGE_SIZE1, IMAGE_SIZE2))
            image = img_to_array(image)
            X_data.append(image)
            y_data.append(class_idx)
    return np.array(X_data), np.array(y_data)

# Load the TFLite model
tflite_model_path = '/content/drive/MyDrive/GaiAppModelPath/GaiApp_Epoch10.tflite'
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

# Load the datasets
X_train, y_train = load_dataset(root_path=train_dir, class_names=class_names, trim=700)
X_val, y_val = load_dataset(root_path=f"{root_dir}valid/", class_names=class_names)
X_test, y_test = load_dataset(root_path=f"{root_dir}test/", class_names=class_names)

# Normalize pixel values
X_train = X_train / 255.0
X_val = X_val / 255.0
X_test = X_test / 255.0

# Evaluate the TFLite model on training dataset
train_accuracy = 0
num_train_samples = len(X_train)

for i in range(num_train_samples):
    input_data = X_train[i].reshape(1, IMAGE_SIZE1, IMAGE_SIZE2, 3).astype(np.float32)
    expected_output = np.array([y_train[i]])

    interpreter.set_tensor(interpreter.get_input_details()[0]['index'], input_data)
    interpreter.invoke()
    output = interpreter.get_tensor(interpreter.get_output_details()[0]['index'])

    predicted_class = np.argmax(output)
    if predicted_class == expected_output[0]:
        train_accuracy += 1

train_accuracy /= num_train_samples
print(f"Training accuracy: {train_accuracy:.4f}")

# Evaluate the TFLite model on validation dataset
val_accuracy = 0
num_val_samples = len(X_val)

for i in range(num_val_samples):
    input_data = X_val[i].reshape(1, IMAGE_SIZE1, IMAGE_SIZE2, 3).astype(np.float32)
    expected_output = np.array([y_val[i]])

    interpreter.set_tensor(interpreter.get_input_details()[0]['index'], input_data)
    interpreter.invoke()
    output = interpreter.get_tensor(interpreter.get_output_details()[0]['index'])

    predicted_class = np.argmax(output)
    if predicted_class == expected_output[0]:
        val_accuracy += 1

val_accuracy /= num_val_samples
print(f"Validation accuracy: {val_accuracy:.4f}")

# Evaluate the TFLite model on testing dataset
test_accuracy = 0
num_test_samples = len(X_test)

for i in range(num_test_samples):
    input_data = X_test[i].reshape(1, IMAGE_SIZE1, IMAGE_SIZE2, 3).astype(np.float32)
    expected_output = np.array([y_test[i]])

    interpreter.set_tensor(interpreter.get_input_details()[0]['index'], input_data)
    interpreter.invoke()
    output = interpreter.get_tensor(interpreter.get_output_details()[0]['index'])

    predicted_class = np.argmax(output)
    if predicted_class == expected_output[0]:
        test_accuracy += 1

test_accuracy /= num_test_samples
print(f"Testing accuracy: {test_accuracy:.4f}")


['Apple', 'Banana', 'Grape', 'Mango', 'Strawberry']
5
Training accuracy: 0.9917
Validation accuracy: 0.8667
Testing accuracy: 0.7900


In [ ]:
#============================================================================================